In [29]:
import pandas as pd
import numpy as np
import re
import datetime as dt
from IPython.display import display, HTML
from datasets import load_dataset
display(HTML("<style>.container { width:100% !important; }</style>"))
from google.cloud import bigquery
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/gikok/code/ingka-search-modelling-dev-3a2a4f44d9c5.json'
import time

In [ ]:
client = bigquery.Client()
query = """
# Retrieves all events with click actions (event_action = select_content),their queries, resulting products, and number of queries resulting in specific product
# Retrieves all events with click actions (event_action = select_content),their queries, and resulting products for the first PLP

WITH session_target AS (
  SELECT
    hit_sequence_number,
    event_category,
    event_action,
    session_id,
    page_url_query_parameter,
    next_page_url,
    SPLIT(REGEXP_EXTRACT(page_url_query_parameter, r'\?q=(\w.+)+'), '&')[SAFE_OFFSET(0)] AS clean_query,
    ARRAY_REVERSE(SPLIT(next_page_url,'/'))[SAFE_OFFSET(0)] AS target_product,
    ARRAY_REVERSE(SPLIT(ARRAY_REVERSE(SPLIT(next_page_url,'/'))[SAFE_OFFSET(0)], '-'))[SAFE_OFFSET(0)] AS target_product_id,
  FROM
    ingka-web-analytics-prod.web_data_v2.hits_events_and_pages
  WHERE
    ( date_hit BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 365 DAY) AND CURRENT_DATE()
      AND website_market_short = 'gb'
      AND event_category = 'engagement'
      AND event_action = 'select_content'
      AND page_url_query_parameter IS NOT NULL) 
      

)
SELECT distinct target_product_id FROM session_target
  WHERE
    clean_query IS NOT NULL
    AND REGEXP_CONTAINS(target_product, r'[0-9A-Za-z]*[\d]')--target_product_id NOT IN ('products', 'en', 'Site Exit', 'gallery', 'shoppingcart', 'latest', 'search', 'login') -- prevent the target_product to become one of these 'next_pages'
"""
n = client.query(query).to_dataframe()

In [ ]:
client = bigquery.Client()
query = """
# Retrieves all events with click actions (event_action = select_content),their queries, resulting products, and number of queries resulting in specific product
# Retrieves all events with click actions (event_action = select_content),their queries, and resulting products for the first PLP

WITH session_target AS (
  SELECT
    hit_sequence_number,
    event_category,
    event_action,
    session_id,
    page_url_query_parameter,
    next_page_url,
    SPLIT(REGEXP_EXTRACT(page_url_query_parameter, r'\?q=(\w.+)+'), '&')[SAFE_OFFSET(0)] AS clean_query,
    ARRAY_REVERSE(SPLIT(next_page_url,'/'))[SAFE_OFFSET(0)] AS target_product,
    ARRAY_REVERSE(SPLIT(ARRAY_REVERSE(SPLIT(next_page_url,'/'))[SAFE_OFFSET(0)], '-'))[SAFE_OFFSET(0)] AS target_product_id,
  FROM
    ingka-web-analytics-prod.web_data_v2.hits_events_and_pages
  WHERE
    ( date_hit BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 365 DAY) AND CURRENT_DATE()
      AND website_market_short = 'gb'
      AND event_category = 'engagement'
      AND event_action = 'select_content'
      AND page_url_query_parameter IS NOT NULL) 

),
clean_query_product AS (
  SELECT
    session_id,
    clean_query,
    REGEXP_REPLACE(target_product, r'-', ' ') AS target_product,
    target_product_id,
    COUNT(*) AS query_n_resulting_product
  FROM
    session_target
  WHERE
    clean_query IS NOT NULL
    AND REGEXP_CONTAINS(target_product, r'[0-9A-Za-z]*[\d]')--target_product_id NOT IN ('products', 'en', 'Site Exit', 'gallery', 'shoppingcart', 'latest', 'search', 'login') -- prevent the target_product to become one of these 'next_pages'
  GROUP BY
    clean_query,
    target_product_id,
    target_product,
    session_id
  ORDER BY
    clean_query
)

SELECT 
  clean_query, 
  ARRAY_AGG(target_product_id IGNORE NULLS order by query_n_resulting_product desc) as term_frequency_ranking
FROM clean_query_product 
GROUP BY clean_query

"""
r = client.query(query).to_dataframe()

In [ ]:
r['rank_1'] = r['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[0]).zfill(8))
r['rank_2'] = r['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[1]).zfill(8) if len(list(x))>1 else None)
r['rank_3'] = r['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[2]).zfill(8) if len(list(x))>2 else None)
r['rank_4'] = r['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[3]).zfill(8) if len(list(x))>3 else None)
r['rank_5'] = r['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[4]).zfill(8) if len(list(x))>4 else None)
r['rank_6'] = r['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[5]).zfill(8) if len(list(x))>5 else None)
r['rank_7'] = r['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[6]).zfill(8) if len(list(x))>6 else None)
r['rank_8'] = r['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[7]).zfill(8) if len(list(x))>7 else None)
r['rank_9'] = r['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[8]).zfill(8) if len(list(x))>8 else None)
r['rank_10'] = r['term_frequency_ranking'].apply(lambda x: re.sub("[^0-9]", "",list(x)[9]).zfill(8) if len(list(x))>9 else None)
r.drop(labels='term_frequency_ranking', axis=1, inplace=True)
r['clean_query'] = r['clean_query'].apply(lambda x: x.replace('%20', ' '))
r['clean_query'] = r['clean_query'].apply(lambda x: x.replace('+', ' '))
r

In [ ]:
r.to_hdf('year_queries.hdf5', index=False, key='year_queries')

In [ ]:
queries

In [ ]:
df = pd.read_csv('descriptions.csv')
df

In [ ]:
df['item_no'] = df['id'].apply(lambda x: re.sub("[^0-9]", "",x.split(",")[1]).zfill(8))
df

In [ ]:
new_df = df[['item_no']+list(df.columns[1:-1])]
new_df

In [ ]:
new_df.to_hdf('all_en_descriptions.hdf5', key='all')

In [ ]:
b.to_hdf('filtered_GB_en_descriptions.hdf5', key='all')

In [ ]:
b = new_df.merge(f, how='inner', on='item_no')

In [ ]:
df.nunique()

NameError: name 'df' is not defined

In [ ]:
t = pd.DataFrame({'item_no':[], 'input':[], 'target':[], 'options':[]})

In [ ]:
t = t.append({'item_no':item_no, 'input':inp, 'target':target, 'options':opt}, ignore_index=True)
t

In [ ]:
functions = [no_to_name, name_to_no, is_description, is_summary, true_query, true_query, true_query, query_rank, query_rank]


In [ ]:
ls -t

In [ ]:
np.random.seed(12)
#queries = pd.read_hdf('year_queries.hdf5')
#df = pd.read_hdf("filtered_GB_en_descriptions.hdf5")
functions = [no_to_name, name_to_no, is_description, is_summary, true_query, true_query, true_query, query_rank, query_rank]
prompt_df = pd.DataFrame({'global_index':[],'item_no':[], 'input':[], 'target':[], 'options':[]})
for i in range(len(df)):
    start = time.time()
    print('iteration: ', i)
    index = i
    item_no = df['item_no'].iloc[index]
    query_df = queries[queries.isin([item_no]).any(axis=1)]
    r = np.random.randint(0, 9, size=1024)
    inputs = []
    targets = []
    options = []
    for j in range(1024):
        f = functions[r[j]]
        
        if r[j] <4:
            inp, tar, opt = f(index)
            inputs += [inp]
            targets += [tar]
            options += [opt]
        
        else:
            inp, tar, opt = f(query_df, item_no)
            inputs += [inp]
            targets += [tar]
            options += [opt]
        
    prompt_df = prompt_df.append(pd.DataFrame({'global_index':list(np.arange(1024*i, 1024*(i+1))),'item_no':1024*[item_no], 'input':inputs, 'target':targets, 'options':options}), ignore_index=True)
    print(f"generating prompts for item_no {item_no} took {np.round(time.time()-start)} seconds.")
    print("")
    
prompt_df.to_hdf('prompts.hdf5', key='alpha')
prompt_df['insertion_time'] = dt.datetime.now()
prompt_df['options'] = prompt_df['options'].apply(lambda x: str(x))
prompt_df['global_index'] = prompt_df['global_index'].astype(int)
prompt_df.to_gbq(destination_table = 'prompts.prompts_001', project_id='ingka-search-modelling-dev', if_exists='append')

In [5]:
df = pd.read_hdf('prompts.hdf5')

In [11]:
df

,global_index,item_no,input,target,options
0,0,10462236,"if item_no is 10462236, which of the following...",hemlagad frying pan black,"['hemlagad frying pan black', 'frasera whiske..."
1,1,10462236,"if name is hemlagad frying pan black, what ite...",10462236,"['60435537', '10462236', '49454495']"
2,2,10462236,query:'32cm fry pan'\nthe query above returns ...,2,"[2, 9, 5, 3, 10]"
3,3,10462236,"if name is hemlagad frying pan black, what ite...",10462236,"['20365094', '39290701', '40221047', '10462..."
4,4,10462236,query:'hemlagad wok'\nthe query above returns ...,3,"[10, 9, 3, 7]"
...,...,...,...,...,...
6180859,6180859,99388356,no need to drill – just click on the handle wh...,no,"[yes, no]"
6180860,6180860,99388356,"rug, low pile. is the previous sentence a summ...",no,"[yes, no]"
6180861,6180861,99388356,storage combination. is the previous sentence ...,yes,"[yes, no]"
6180862,6180862,99388356,storage combination. is the previous sentence ...,yes,"[yes, no]"


In [9]:
df.to_parquet('df.parquet.gzip',
              compression='gzip')

In [31]:
df = pd.read_parquet('df.parquet.gzip')

FileNotFoundError: [Errno 2] No such file or directory: 'df.parquet.gzip'

In [40]:
df = pd.read_hdf("../data/filtered_GB_en_descriptions.hdf5")
d = df['item_no'].drop_duplicates().reset_index(drop=True)
pd.DataFrame(d).to_parquet('../data/item_no_6k.gzip')

In [15]:
data = load_dataset('.', data_files='df.parquet.gzip')

Using custom data configuration .-7e90710a90d454b2


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /Users/gikok/.cache/huggingface/datasets/parquet/.-7e90710a90d454b2/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [28]:
def tokenize_train(examples):
    input_texts = examples['input']
    target_texts = examples['target']
    opts = examples['options']
    print(input_texts[0])
    print(type(input_texts), len(input_texts))
    print("lol")
    return input_texts
train_dataset = data.map(
    tokenize_train, batched=True
)

  0%|          | 0/6181 [00:00<?, ?ba/s]

if item_no is 10462236, which of the following is the correct name: 'hemlagad frying pan black' or 'frasera whiskey glass'?
["'hemlagad frying pan black'", " 'frasera whiskey glass'"]
<class 'list'> 1000
lol


TypeError: Provided `function` which is applied to all elements of table returns a variable of type <class 'list'>. Make sure provided `function` returns a variable of type `dict` (or a pyarrow table) to update the dataset or `None` if you are only interested in side effects.

In [ ]:
start = time.time()
inp, target, opt = no_to_name(index)
item_no = df['item_no'].iloc[0]
print("################# PROMPT TYPE 1 ########################")
print("INPUT:",inp)
print("")
print("")
print("TARGET:", target)
inp, target, opt = name_to_no(index)
print("################# PROMPT TYPE 2 ########################")
print("INPUT:",inp)
print("")
print("")
print("TARGET:", target)
inp, target, opt = is_description(index)
print("################# PROMPT TYPE 3 ########################")
print("INPUT:",inp)
print("")
print("")
print("TARGET:", target)
inp, target, opt = is_summary(index)
print("################# PROMPT TYPE 4 ########################")
print("INPUT:",inp)
print("")
print("")
print("TARGET:", target)   
# get queries that have item_no
query_df = queries[queries.isin([item_no]).any(axis=1)]
inp, target, opt = true_query(query_df, item_no)
print("################# PROMPT TYPE 5 ########################")
print("INPUT:",inp)
print("")
print("")
print("TARGET:", target)
inp, target, opt = query_rank(query_df, item_no)
print("################# PROMPT TYPE 6 ########################")
print("INPUT:",inp)
print("")
print("")
print("TARGET:", target)
end = time.time()
print(f"It took {(end-start)*1e3} ms to generate prompts")


In [ ]:
type(np.arange(3)[0])

In [ ]:
n[n['item_no']=='00248567']

In [ ]:
def add_or(s):
    li = s.rsplit(', ')
    start = ', '.join(li[:-1])
    end =  ' or ' + li[-1]
    return start+end
    

def create_option_string(df, column, inds):
    options = ""
    for i in inds:
        options +=  f"'{df[column].iloc[i]}', "
        if i == inds[-1]:
            options = options[:-2]
            
    option_string = add_or(options)
    option_list = options.split(',')
        
    return option_string, option_list

def get_other_indices(index, size, num):
    """
    given a number {index}, returns {num} integers in range({size})
    """
    
    proceed = 0
    while proceed == 0:
        inds = np.random.choice(range(size), num)
        # check if any of the new indices is same as initial
        if (inds == index).any()==False:
            proceed = 1
        else:
            proceed = 0
    
    return inds


def no_to_name(index):
    
    # get random number of options
    num = np.random.randint(1, 4)
    
    # get random item index
    inds = get_other_indices(index, len(df), num)
    inds = np.append(inds, index)
    np.random.shuffle(inds)
    
    # get answer options
    options, option_list = create_option_string(df, 'name', inds)
    
    # create the input string
    inp = f"if item_no is {df['item_no'].iloc[index]}, which of the following is the correct name: " + options + '?'
    
    
    # set the target
    target = df['name'].iloc[inds[np.argmin(inds-index)]]
    
    return inp.lower(), target.lower(), option_list

def name_to_no(index):
    
    # get random number of options
    num = np.random.randint(1, 4)
    
    # get random item index
    inds = get_other_indices(index, len(df), num)
    inds = np.append(inds, index)
    np.random.shuffle(inds)
    
    # get answer options
    options, option_list = create_option_string(df, 'item_no', inds)
    
    # get the input string
    inp = f"if name is {df['name'].iloc[index]}, what item_no does it refer to? " + options + '?' 
    
    # get the target
    target = df['item_no'].iloc[inds[np.argmin(inds-index)]]
    
    return inp.lower(), target.lower(), option_list

def is_description(index):
    
    is_true = np.random.randint(2)
    
    if is_true:
        desc = df['benefits'].iloc[index]
        target = "yes"
    else:
        tempdf = df[df['benefits']!=df['benefits'].iloc[index]]
        desc = np.random.choice(tempdf['benefits'])
        target = "no"
        
    # make sure ends with period    
    desc = desc if desc.endswith(".") else desc + "."
    
    # create input
    inp = desc + f" is the previous sentence a description of item_no {df['item_no'].iloc[index]}. yes or no?"
    
    return inp.lower(), target, ['yes', 'no']

def is_summary(index):
    
    is_true = np.random.randint(2)
    
    if is_true:
        desc = df['key_w'].iloc[index]
        target = "yes"
    else:
        tempdf = df[df['key_w']!=df['key_w'].iloc[index]]
        desc = np.random.choice(tempdf['key_w'])
        target = "no"
        
    # make sure ends with period    
    desc = desc if desc.endswith(".") else desc + "."
    
    # create input
    inp = desc + f" is the previous sentence a summary of item_no {df['item_no'].iloc[index]}. yes or no?"
    
    return inp.lower(), target, ['yes', 'no']

def true_query(query_df, item_no):
    
    is_true = np.random.randint(2)
    
    if is_true:
        query = query_df['clean_query'].sample().iloc[0]
        target = "yes"
    else:
        query = query_df['clean_query'].sample().iloc[0]
        target = "no"

    
    # create input
    inp = "query:'" + query + f"'\ndoes the query above return item_no {item_no} as a result. yes or no?"
    
    return inp.lower(), target, ['yes', 'no']

def query_rank(query_df, item_no):

    # select random row
    row = query_df.sample()
    
    # get query string
    query = row['clean_query'].iloc[0]
    
    # get rank of item_no
    cols = np.array(row.columns)
    inds = (row[row==item_no].isnull()==False).values[0]
    
    # get first true value, that is not 0 (queries can be for item_no) and
    # sometimes same item_no has multiple ranks
    if len(inds)>1:
        inds[0] = False
        rank = cols[np.argmax(inds)][-1]
        
        # if doesn't work keep trying
        c = 0
        while rank.isdigit()==False:
            row = query_df.sample()
            inds = (row[row==item_no].isnull()==False).values[0]
            inds[0] = False
            rank = cols[np.argmax(inds)][-1]
            c +=1
            if c>100:
                print(f'stuck in infinite loop with item_no {item_no}')
                break
            
    
    # in case rank is 10
    rank = '10' if rank=='0' else rank
    
    # create possible options
    n_options = np.random.randint(1, 5)
    
    options = list(np.arange(1, 11))
    options.remove(int(rank))
    
    options = np.append(np.random.choice(options, n_options).astype(str), rank)
    np.random.shuffle(options)
    
    # create input
    inp = "query:'" + query + f"'\nthe query above returns item_no {item_no} as a result. what is its rank? " + add_or(', '.join(options))
    
    target = rank
    return inp.lower(), target, list(options)

In [ ]:
l = '1'
l.isdigit()

In [ ]:
df.drop(labels = 'term_frequency_ranking', axis = 1, inplace=True)

In [ ]:
n['item_no'] = n['target_product_id'].copy()

In [ ]:
n['item_no'] = n['target_product_id'].apply(lambda x: re.sub("[^0-9]", "",str(x)).zfill(8))

In [ ]:
f = pd.DataFrame()

In [ ]:
f['item_no'] = pd.unique(queries[queries.columns[1:]].values.ravel('K'))

In [ ]:
prompts

In [ ]:
t.apply({'test':'test', 'testtest':"testtest"})

In [ ]:
raw_train_dataset['train']

In [ ]:
raw_train_dataset = load_dataset('super_glue', 'cb', split='train')
prompts = DatasetTemplates('super_glue', "cb")
template = prompts['consider always/sometimes/never']

column_names = raw_train_dataset.column_names
def preprocess_train(examples):
    bs = len(examples[column_names[0]])

    input_texts = []
    target_texts = []
    answer_choices_texts = []
    for i in range(10):
        ex = {
            k: examples[k][i]
            for k in column_names
        }
        input, target = template.apply(ex)
        ex_answer_choices = template.get_answer_choices_list(ex)
        assert target in ex_answer_choices
        input_texts.append(input)
        target_texts.append(target)
        answer_choices_texts.append(ex_answer_choices)
    return input_texts, target_texts, answer_choices_texts

i, t, a = preprocess_train(raw_train_dataset)
for j in range(10):
    print("%r"%i[j])
    print("%r"%t[j])
    print("%r"%a[j])
    print("********************")

In [ ]:
# get queries that have item_no
qs = queries[queries.isin([item_no]).any(axis=1)]

# select random row
row = qs.sample()

# get query string
query = queries['clean_query']

# get rank of item_no
cols = np.array(row.columns)
inds = (row[row==item_no].isnull()==False).values[0]

In [ ]:
item_no

In [ ]:
row

In [ ]:
inds

In [ ]:
item_no